![BTS](https://4d17d13o8yuu3wluzr4c2feo-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/btslogo.png)

### Master in Big Data Solutions Final Project - Smart Tour

#### Pablo Dellacassa, Santiago Borgnino & Muhannad Shahada


Data preparation

### Data Sources



1. Number of tourists in Catalunya per month from 2015 to 2019:
https://datos.gob.es/es/catalogo/ea0010587-numero-de-turistas-segun-comunidad-autonoma-de-destino-principal-mensual-comunidades-autonomas-movimientos-turisticos-en-fronteras-identificador-api-108231

2. Barcelona list of tourist attractions and number of annual visitors from 1994 to 2019: Observatorio de Turismo de Barcelona

3. Barcelona list of gastronomic equipment: https://opendata-ajuntament.barcelona.cat/data/es/dataset/equipament-restaurants

4. Barcelona tourist attractions geolocation: https://opendata-ajuntament.barcelona.cat/data/es/dataset/punts-informacio-turistica

5. Lonely Planet list of Barcelona tourist attractions with features: Web Scraping from Web Site
https://www.lonelyplanet.com/spain/barcelona/attractions?page=1

6. TripAdvisor Restaurants Info for 31 Euro-Cities: https://www.kaggle.com/damienbeneschi/krakow-ta-restaurans-data-raw

7. Barcelona neighbourhoods and districs geodata: https://github.com/martgnz/bcn-geodata



### Package import

In [2]:
import pandas as pd
from re import search
from math import sqrt, radians
from sklearn.neighbors import DistanceMetric
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Loading datasets

In [2]:
#od is OpenData, pits is Points of Interest
pits_od=pd.read_excel('pits_opendata.xls')

#ob is Observsatorium de Turismo de Barcelona, pits is Points of Interest
pits_ob=pd.read_excel('barcelona-cultura-y-ocio-1994-2019.xlsx')

#TA is TripAdvisor
TA_data=pd.read_csv('TA_restaurants_curated.csv')

#lp is Lonely Planet
pits_lp=pd.read_excel('pits_lp.xlsx', sheet_name='Curado')

restaurants_op=pd.read_csv('restaurants_opendata.csv')

#cat is Catalunya
cat_visitors=pd.read_excel('catalunya_visitors2015-2019.xlsx')

#POI_barcleona
poi_bcn=pd.read_excel('POI_barcelona.xlsx')

#### Formatting, dropping columns, identifying missing values

##### Catalunya visitors dataset

In [3]:
cat_visitors.head(2)

,Año,Mes,Mes2,Dato base,Tasa de variación anual,Acumulado en lo que va de año,Tasa de variación acumulada,Total por año,Porcentaje
0,2019,12,December,993169,2.46,19375152,0.93,19375153,0.051260
1,2019,11,November,1047153,1.21,18381984,0.85,19375153,0.054046


In [4]:
cat_visitors.shape

(51, 9)

In [5]:
cat_visitors.dtypes

Año                                        int64
Mes                                        int64
Mes2                                      object
        Dato base                          int64
        Tasa de variación anual          float64
        Acumulado en lo que va de año     object
        Tasa de variación acumulada       object
Total por año                              int64
Porcentaje                               float64
dtype: object

In [6]:
cat_visitors.columns

Index(['Año', 'Mes', 'Mes2', '        Dato base',
       '        Tasa de variación anual',
       '        Acumulado en lo que va de año',
       '        Tasa de variación acumulada', 'Total por año', 'Porcentaje'],
      dtype='object')

In [7]:
#Removing space from benining and end of columns names
cat_visitors.columns = cat_visitors.columns.str.lstrip()

In [8]:
cat_visitors.columns

Index(['Año', 'Mes', 'Mes2', 'Dato base', 'Tasa de variación anual',
       'Acumulado en lo que va de año', 'Tasa de variación acumulada',
       'Total por año', 'Porcentaje'],
      dtype='object')

In [9]:
#Dropping unnecessary columns
cat_visitors=cat_visitors.drop(['Mes2', 'Tasa de variación anual', 'Acumulado en lo que va de año', 'Tasa de variación acumulada'], axis=1)

In [10]:
cat_visitors.head()

,Año,Mes,Dato base,Total por año,Porcentaje
0,2019,12,993169,19375153,0.051260
1,2019,11,1047153,19375153,0.054046
2,2019,10,1676430,19375153,0.086525
3,2019,9,2011704,19375153,0.103829
4,2019,8,2363331,19375153,0.121977


##### Points of interest Observatorium of Tourism in Barcelona

In [11]:
pits_ob.head(2)

,numero,Visitantes de museos y colecciones (MC),Type,1994,2000,2011,2014,2015,2016,2017,2018,2019
0,1,Museu FC Barcelona,Museums and Collections,538077,1156090,0,1530484,1785903,1947014,1848198,1730335,1661156
1,2,L'Aquàrium de Barcelona,Museums and Collections,0,1563493,0,1590420,1549480,1587828,1626193,1631108,1609373


In [12]:
pits_ob.dtypes

numero                                      int64
Visitantes de museos y colecciones (MC)    object
Type                                       object
1994                                        int64
2000                                        int64
2011                                        int64
2014                                        int64
2015                                       object
2016                                       object
2017                                        int64
2018                                        int64
2019                                        int64
dtype: object

In [13]:
#Dropping unnecessary columns with index number
pits_ob=pits_ob.drop('numero', axis=1)

In [14]:
#Replace string values with 0 in columns 2015 and 2016
pits_ob[2015].replace(['Cerrado', 'nd'], 0, inplace=True)
pits_ob[2016].replace(['Cerrado', 'nd'], 0, inplace=True)

In [15]:
#Changing columns 2015 and 2016 to int type
pits_ob[2015]=pits_ob[2015].astype(int)
pits_ob[2016]=pits_ob[2016].astype(int)

In [16]:
#Renaming columns
pits_ob.rename(columns={'Visitantes de museos y colecciones (MC)':'name', 'Type':'type'}, inplace=True)

In [17]:
#Apply upper case for names
pits_ob['name'] = pits_ob['name'].str.upper()
pits_ob['type']= pits_ob['type'].str.upper()

#Remove white space at the beginning and end of the name
pits_ob['name'] = pits_ob['name'].str.lstrip()
pits_ob['type']= pits_ob['type'].str.lstrip()

In [18]:
#Check for duplicate names
pits_ob.name.value_counts()

MUSEU TORRE BALLDOVINA - SANTA COLOMA DE GRAMANET                     1
MUSEU D’IDEES I INVENTS DE BARCELONA. MIBA (4)                        1
LA PEDRERA                                                            1
IMAX(1)                                                               1
CENTRE D'INTERPRETACIÓ DEL CAVA-CIC FASSINA (SANT SADURNÍ D'ANOIA)    1
                                                                     ..
PARC NATURAL DE LA SERRA DE COLLSEROLA                                1
PALAU GÜELL                                                           1
MUSEU DE LA MOTO                                                      1
PARC D'ATRACCIONS TIBIDABO                                            1
MUSEU DE LA COLÒNIA VIDAL (PUIG-REIG)                                 1
Name: name, Length: 134, dtype: int64

In [19]:
#Checking every column has a value
pits_ob.count()

name    134
type    134
1994    134
2000    134
2011    134
2014    134
2015    134
2016    134
2017    134
2018    134
2019    134
dtype: int64

##### Points of interest Open Data Barcelona

In [20]:
pits_od.dtypes

pos                    int64
num                    int64
address               object
city                  object
code_url              object
district              object
atencio_eq            object
gmapx                float64
gmapy                float64
id                     int64
name                  object
phonenumber           object
type                  object
tp                    object
title                 object
excerpt               object
content               object
date                  object
author                object
categories           float64
tags                 float64
language              object
originalpost           int64
related_post         float64
attachments          float64
vignette             float64
moreinfo             float64
usergroup             object
post_modified         object
original_modified     object
wt                    object
sigla                 object
sectionname           object
dtype: object

In [21]:
pits_od.shape

(851, 33)

In [22]:
#Drop unnecessary columns
pits_od=pits_od.drop(['originalpost','categories','atencio_eq', 'phonenumber','type','tp','date','author','tags','language','pos', 'num', 'city', 'address', 'code_url', 'related_post', 'attachments', 'vignette', 'moreinfo', 'usergroup', 'post_modified', 'original_modified', 'wt', 'sigla', 'sectionname'], axis=1)

In [23]:
pits_od.head(2)

,district,gmapx,gmapy,id,name,title,excerpt,content
0,Sants-Montjuïc,41.361374,2.159711,190829,Parc de Montjuïc,El parc de Montjuïc,<p>El gran espai natural del parc de Montjuïc ...,<h3>Un turó amb història</h3>\r\n<p>Assentat s...
1,Sants-Montjuïc,41.361374,2.159711,190829,Parc de Montjuïc,El parc de Montjuïc,<p>El gran espai natural del parc de Montjuïc ...,<h3>Un turó amb història</h3>\r\n<p>Assentat s...


In [24]:
#Apply upper case for names
pits_od['name'] = pits_od['name'].str.upper()

#Remove white space at the beginning and end of the name
pits_od['name'] = pits_od['name'].str.lstrip()


In [25]:
#Check for duplicates
pits_od.id.value_counts()

190829    2
286714    1
285228    1
191031    1
190989    1
         ..
541754    1
284089    1
541751    1
285749    1
541205    1
Name: id, Length: 850, dtype: int64

In [26]:
#Drop duplicate values
pits_od=pits_od.drop_duplicates('id')
pits_od.head(2)

,district,gmapx,gmapy,id,name,title,excerpt,content
0,Sants-Montjuïc,41.361374,2.159711,190829,PARC DE MONTJUÏC,El parc de Montjuïc,<p>El gran espai natural del parc de Montjuïc ...,<h3>Un turó amb història</h3>\r\n<p>Assentat s...
2,Sant Martí,41.411034,2.221698,190952,EL PARC DEL FÒRUM,El parc del Fòrum,<p>El recinte projectat arran de mar per acoll...,<h3>Un gran esdeveniment</h3>\r\n<p>El parc de...


In [27]:
#Removing noise strings from exceprt
pits_od['excerpt'] = pits_od['excerpt'].str.replace('<p>', '')
pits_od['excerpt'] = pits_od['excerpt'].str.replace('</p', '')


#Removing noise strings from content
pits_od['content']=pits_od['content'].str.replace('<h3>', '' )
pits_od['content']=pits_od['content'].str.replace('</h3>', '' )
pits_od['content']=pits_od['content'].str.replace('\r\n', '' )
pits_od['content']=pits_od['content'].str.replace('<p>', '' )
pits_od['content']=pits_od['content'].str.replace('<strong>', '' )
pits_od['content']=pits_od['content'].str.replace('</strong>', '' )
pits_od['content']=pits_od['content'].str.replace('</p>', '' )
pits_od['content']=pits_od['content'].str.replace('<!-- .photo-galleria -->', '' )

print(pits_od.excerpt[0])
print(" ")
print(pits_od.content[0])

El gran espai natural del parc de Montjuïc és el millor lloc per gaudir de la natura i la cultura alhora, perquè és ple de magnífics jardins i d’instal·lacions culturals.>
 
Un turó amb històriaAssentat sobre el turó que recorre el barri de Sants i mira cap al mar, Montjuïc ha estat testimoni i escenari de múltiples fets transcendentals en la història de Barcelona. Es va començar a urbanitzar a partir de l’Exposició Universal del 1929. Després dels successos dramàtics de la Guerra Civil, en què el castell va funcionar com a presó, l’indret va canviar i, amb els Jocs Olímpics de 1992, es va renovar completament i va tornar a adquirir un caràcter festiu i alegre per als barcelonins.El nom del turó, de 177 metres d’alçària, ha estat un tema de controvèrsia, ja que Montjuïc en català medieval podria traduir-se com a &#8216;Mont dels jueus&#8216;, la qual cosa està avalada per l’existència d’un cementiri jueu a la muntanya.Natura, cultura i esportEn aquest gran pulmó verd de la ciutat s’hi 

In [28]:
pits_od.district.value_counts()

Gràcia                 134
Sarrià-Sant Gervasi    119
Ciutat Vella           116
Sant Andreu             95
Sants-Montjuïc          88
Eixample                85
Les Corts               75
Sant Martí              57
Horta-Guinardó          52
Nou Barris              27
Name: district, dtype: int64

In [29]:
pits_od.count()

district    848
gmapx       850
gmapy       850
id          850
name        850
title       850
excerpt     850
content     850
dtype: int64

##### Trip Advisor dataset

In [30]:
TA_data.head(2)

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419


In [31]:
TA_data.shape

(125527, 11)

In [32]:
TA_data.dtypes

Unnamed: 0             int64
Name                  object
City                  object
Cuisine Style         object
Ranking              float64
Rating               float64
Price Range           object
Number of Reviews    float64
Reviews               object
URL_TA                object
ID_TA                 object
dtype: object

In [33]:
TA_data_bcn=TA_data[TA_data['City']=='Barcelona']

In [34]:
TA_data_bcn.shape

(8425, 11)

In [35]:
TA_data_bcn.head(2)
TA_data_bcn=TA_data_bcn.drop(['Unnamed: 0', 'URL_TA', 'ID_TA', 'City'], axis=1)

In [36]:
TA_data_bcn.replace('NaN', np.nan, inplace=True)
TA_data_bcn.columns= TA_data_bcn.columns.str.lower()

In [37]:
TA_data_bcn.head(2)

,name,cuisine style,ranking,rating,price range,number of reviews,reviews
5372,Uma,"['International', 'Mediterranean', 'Fusion', '...",1.0,5.0,$$$$,792.0,"[['Perfect atmosphere and location', 'Perfecti..."
5373,Viana,"['International', 'Mediterranean', 'Spanish', ...",2.0,5.0,$$ - $$$,2707.0,"[['Wow! Best ever!', 'Small menu-- GET A RESER..."


In [38]:
TA_data_bcn=TA_data_bcn.reset_index().drop('index', axis=1)

In [39]:
TA_data_bcn['price range'].value_counts().sum()

5407

In [40]:
TA_data_bcn.count()

name                 8425
cuisine style        6388
ranking              7795
rating               7793
price range          5407
number of reviews    7264
reviews              7793
dtype: int64

In [41]:
TA_data_bcn.shape

(8425, 7)

In [42]:
pits_od=pits_od.drop_duplicates()

In [43]:
pits_od[pits_od['name']=='El Parc del Fòrum']['title']

Series([], Name: title, dtype: object)